<a href="https://colab.research.google.com/github/ZoryAce/Topicos/blob/main/Taller4_PretrainedModelsPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tranfer Learnning to Flower Recognition using PyTorch 🔥
This dataset contains labeled 4242 images of flowers.


### Content
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.

For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. Photos are not reduced to a single size, they have different proportions!

## Data
You can download data from: [Flowers Recognition Dataset](https://www.kaggle.com/datasets/alxmamaev/flowers-recognition)

In [ ]:
import numpy as np
import pandas as pd
import os
path="./"
os.listdir(path)

In [ ]:
!unzip flowers.zip

Archive:  flowers.zip
replace flowers/daisy/10140303196_b88d3d6cec.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
path="./flowers"
os.listdir(path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
classes=['dandelion', 'daisy', 'sunflower', 'tulip', 'rose']

In [ ]:
#CONTEO DE IMAGENES EN EL ARCHIVO LA CUAL DIFIERE DE LO INDICADO
#EN EL ENCABEZADO (4317 IMAGENES)
img_type = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
img_qty = sum([len(files) for r, d, files in os.walk(path)
                   if any(file.lower().endswith(img_type) for file in files)])

print(f"Número de imagenes: {img_qty}")

In [ ]:
label_of_file=[]
img_list=[]

for kind in classes:
    kind_parh=os.path.join(path,kind)

    for img in os.listdir(kind_parh):
        img_list.append(os.path.join(kind_parh,img))
        label_of_file.append(kind)

df=pd.DataFrame({'img':img_list,'label':label_of_file})
df.head(3)
df = df.sample(frac=1).reset_index(drop=True)
df.head(2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df['encode_label'] = encoder.fit_transform(df['label'])

In [ ]:
df.head(10)

In [ ]:
!pip install opencv-python
import cv2
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torchvision import transforms, models

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.imshow(example_img)
plt.show()

In [ ]:
example_img = cv2.imread(df['img'][2])
example_img.shape

In [ ]:
example_img_rgb = cv2.cvtColor(example_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(example_img_rgb)
plt.axis('off')  # Opcional: oculta los ejes
plt.show()


In [ ]:
import numpy as np
x=[]
for img in df['img']:
    try:
        img = cv2.imread(img)
        img = cv2.resize(img, (150, 150))
        img = img / 255.0  # normalize
        x.append(img)      # img

    except:
        print(f"Error loading image: {img}")

## Modeling

In [ ]:

# Convertir a tensores x y y
X_tensor = torch.tensor(np.array(x).transpose(0, 3, 1, 2), dtype=torch.float32)  # Formato (N, C, H, W)
y_tensor = torch.tensor(df['encode_label'].values, dtype=torch.long)              # Etiquetas como tensor

In [ ]:
df_tensor = TensorDataset(X_tensor, y_tensor)#crear dataset de tensores

In [ ]:
#Creacion dataset entrenamiento y pruebas
train_size = int(0.8 * len(df_tensor))
test_size = len(df_tensor) - train_size
train_dataset, test_dataset = random_split(df_tensor, [train_size, test_size])

In [ ]:
# Creación de dataloader de entrenamiento y pruebas
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# make model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 5)


In [ ]:
# train Model
criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# train model
def train_model(model, criterion, optimizer, num_epochs=25):
  model.train()

  for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in zip(train_loader):
      optimizer.zero_grad()
      inputs = inputs.double()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item() * inputs.size(0)
      avg_loss = running_loss / len(train_loader.dataset)
      print(f"Época {epoch+1}/{num_epochs} - Pérdida: {avg_loss:.4f}")
    print("Entrenamiento completado ✅")

In [ ]:
# train model chat
def train_model(model, criterion, optimizer, num_epochs=25):
  model.train()

  for epoch in range(num_epochs):
    running_loss = 0.0
    # Iterate directly over the train_loader
    for inputs, labels in train_loader:
      optimizer.zero_grad()
      #inputs = inputs.double()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item() * inputs.size(0)
      avg_loss = running_loss / len(train_loader.dataset)
      print(f"Época {epoch+1}/{num_epochs} - Pérdida: {avg_loss:.4f}")
    print("Entrenamiento completado ✅")

In [ ]:
# Test Model
def test_model(model):
  model.eval()
  test_loss = 0.0
  correct = 0
  with torch.no_grad():
    for inputs, labels in zip(test_loader):
      # Permute the dimensions of the input tensor
      inputs = inputs.permute(0, 3, 1, 2) # Change the order of dimensions from [batch_size, height, width, channels] to [batch_size, channels, height, width]
      outputs = model(inputs)
      test_loss += criterion(outputs, labels).item() * inputs.size(0)
      pred = outputs.argmax(dim=1, keepdim=True)
      correct += pred.eq(labels.view_as(pred)).sum().item()
  test_loss /= len(X_train.dataset)
  test_accuracy = 100. * correct / len(X_train.dataset)
  print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(X_train.dataset)} ({test_accuracy:.0f}%)')

In [ ]:
#chat
def test_model(model, test_loader, criterion):
    model.eval()  # Modo evaluación
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for inputs, labels in test_loader:  # Iteración correcta
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item() * inputs.size(0)

            pred = outputs.argmax(dim=1, keepdim=True)  # Predicción más probable
            correct += pred.eq(labels.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print(f'Test set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} ({test_accuracy:.0f}%)')


In [ ]:
torch.cuda.is_available()

In [ ]:
# training
train_model(model, criterion, optimizer, num_epochs=20)

In [ ]:
model.eval()

In [ ]:
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():  # Desactiva el cálculo de gradientes
    for images, labels in test_loader:
        #images, labels = images.to(device), labels.to(device)  # GPU si está disponible
        outputs = model(images)

        # Calcular la pérdida
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Calcular precisión
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

# Mostrar resultados
print(f'Pérdida en test: {test_loss / len(test_loader):.4f}')
print(f'Precisión en test: {100 * correct / total:.2f}%')
